# Split Data

In [1]:
!pip install rank_bm25

In [2]:
import torch 
import numpy as np 
from underthesea import word_tokenize
from underthesea import sent_tokenize
from copy import deepcopy
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import json 


/home/thanhdo/anaconda3/envs/former/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("train_sentences.json") as f:
    data =  json.load(f)

In [4]:
train, val_test = sklearn.model_selection.train_test_split(data['data'], train_size=110, random_state=42)
val, test = sklearn.model_selection.train_test_split(data['data'], train_size=14, test_size=14, random_state=42)

In [5]:
train_json = {"version": "train", "data": train}
train_object = json.dumps(train_json, ensure_ascii=False)
with open("train_file.json", "w") as write_file:
    write_file.write(train_object)

val_json = {"version": "val", "data": val}
val_object = json.dumps(val_json, ensure_ascii=False)
with open("val_file.json", "w") as write_file:
    write_file.write(val_object)

test_json = {"version": "test", "data": test}
test_object = json.dumps(test_json, ensure_ascii=False)
with open("test_file.json", "w") as write_file:
    write_file.write(test_object)

# Helper Functions

In [6]:
def find_sentence(corpus, start_answer):
    '''
    Find sentence contains answer
    * corpus: list sentences of context
    * start_answer: the begin position of answer in context
    '''
    start, end = 0, -1
    sent_idx = -1
    for i in range(len(corpus)):
        sentence = corpus[i]
        start = end + 1
        end = start+len(sentence)
        if start <= start_answer < end:
            sent_idx = i
            break
        elif start_answer == end:
            sent_idx = i + 1
            break
    return sent_idx


def accuracy(data, top_k):
    """
    Calculate accuracy 
    Given queries Q = {Q1, ..., Qm} 
    Document D = {S1, ..., Sn}
    The sentence containing the answer to the question Qi is Ai

    Acc@K = 1/|Q| * sum(is Ai in get_top_k(Q, D))
    """
    results = []
    invalid = 0  # number of invalid qa
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0:

                    # If the answer is from multiple sentences then discard this qa
                    if len(sent_tokenize(qa['answers'][0]['text'])) > 1:
                        invalid += 1
                        continue

                    start_answer = qa['answers'][0]['answer_start']
                    k = min(top_k, len(qa['candidate_indices']))

                    # Find the index of sentence that contains the answer
                    ans_sent_idx = find_sentence(context, start_answer)
                    # Check if the above sentence is retrived in top_k
                    results.append(ans_sent_idx in qa['candidate_indices'][:k])
    return {"top_k": top_k, 
            "# valid qa": len(results), 
            "# invalid qa": invalid,  
            "true": sum(results), 
            "accuracy": round(sum(results) / len(results), 3)}


In [7]:
corpus = ['nguyen thanh do.', 'pham lan phuong.']
start_answer = len(corpus[0]) + len(corpus[1])-1
idx = find_sentence(corpus, start_answer)
corpus[idx], idx

('pham lan phuong.', 1)

# Using BM25

In [8]:
from rank_bm25 import BM25Okapi
from rank_bm25 import BM25Plus
def get_topk_sentences_bm25(data, top_k):
    """
    Using sklearn TFIDF to get top k most relevant sentences for each question
    """
    copied_data = deepcopy(data)
    for topic in copied_data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            vectorizer = TfidfVectorizer()
            tokenizer = vectorizer.build_tokenizer()
            tokenized_context = [tokenizer(sentence.lower()) for sentence in context]
            
            # learn statistic of the context sentences.
            bm25 = BM25Plus(tokenized_context)
            for qa in paragraph['qas']:
                question = qa['question']
                query = tokenizer(question)

                # compute scores of each sentence in the context.
                scores = bm25.get_scores(query)
                k = min(top_k, len(context))
                top_results = torch.topk(torch.tensor(scores.reshape(-1)), k=k)
                qa['candidate_indices'] = top_results[1].tolist()
    return copied_data

In [9]:
bm25_train = get_topk_sentences_bm25(train_json, 10)
bm25_val = get_topk_sentences_bm25(val_json, 10)
bm25_test = get_topk_sentences_bm25(test_json, 10)

In [10]:
print("BM25 Train")
bm25_train_results = []
for i in range(10):
    json_i = accuracy(bm25_train, i+1)
    bm25_train_results.append(json_i)
import pandas as pd 
bm25_train_df = pd.read_json(json.dumps(bm25_train_results))
bm25_train_df

BM25 Train


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,15575,86,11576,0.743
1,2,15575,86,13641,0.876
2,3,15575,86,14524,0.933
3,4,15575,86,15001,0.963
4,5,15575,86,15268,0.980
5,6,15575,86,15397,0.989
6,7,15575,86,15485,0.994
7,8,15575,86,15537,0.998
8,9,15575,86,15549,0.998
9,10,15575,86,15559,0.999


In [11]:
print("BM25 Validate")
bm25_val_results = []
for i in range(10):
    json_i = accuracy(bm25_val, i+1)
    bm25_val_results.append(json_i)
import pandas as pd 
bm25_val_df = pd.read_json(json.dumps(bm25_val_results))
bm25_val_df

BM25 Validate


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1871,4,1399,0.748
1,2,1871,4,1626,0.869
2,3,1871,4,1726,0.923
3,4,1871,4,1790,0.957
4,5,1871,4,1834,0.980
5,6,1871,4,1853,0.990
6,7,1871,4,1858,0.993
7,8,1871,4,1863,0.996
8,9,1871,4,1864,0.996
9,10,1871,4,1864,0.996


In [12]:
print("BM25 Test")
bm25_test_results = []
for i in range(10):
    json_i = accuracy(bm25_test, i+1)
    bm25_test_results.append(json_i)
import pandas as pd 
bm25_test_df = pd.read_json(json.dumps(bm25_test_results))
bm25_test_df

BM25 Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1310,0.770
1,2,1701,3,1502,0.883
2,3,1701,3,1598,0.939
3,4,1701,3,1646,0.968
4,5,1701,3,1677,0.986
5,6,1701,3,1687,0.992
6,7,1701,3,1691,0.994
7,8,1701,3,1698,0.998
8,9,1701,3,1700,0.999
9,10,1701,3,1701,1.000


# TFIDF 

In [13]:
import sklearn 
def get_topk_sentences_sklearn(data, top_k):
    """
    Using sklearn TFIDF to get top k most relevant sentences for each question
    """
    copied_data = deepcopy(data)
    for topic in copied_data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            vectorizer = TfidfVectorizer()
            tokenizer = vectorizer.build_tokenizer()

            # learn statistic of the context sentences.
            X = vectorizer.fit_transform(context)
            for qa in paragraph['qas']:
                question = qa['question']
                y = vectorizer.transform([question])
                # scores = np.dot(X, y.T).toarray()

                # compute scores using cosine similarity of each sentence in the context.
                scores = sklearn.metrics.pairwise.cosine_similarity(y, X)
                k = min(top_k, len(X.toarray()))
                top_results = torch.topk(torch.tensor(scores.reshape(-1)), k=k)
                qa['candidate_indices'] = top_results[1].tolist()
    return copied_data

In [14]:
tfidf_train = get_topk_sentences_sklearn(train_json, 10)
tfidf_val = get_topk_sentences_sklearn(val_json, 10)
tfidf_test = get_topk_sentences_sklearn(test_json, 10)

In [15]:
print("TFIDF Train")
tfidf_train_results = []
for i in range(10):
    json_i = accuracy(tfidf_train, i+1)
    tfidf_train_results.append(json_i)
import pandas as pd 
tfidf_train_df = pd.read_json(json.dumps(tfidf_train_results))
tfidf_train_df

TFIDF Train


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,15575,86,11994,0.770
1,2,15575,86,13997,0.899
2,3,15575,86,14774,0.949
3,4,15575,86,15124,0.971
4,5,15575,86,15343,0.985
5,6,15575,86,15447,0.992
6,7,15575,86,15506,0.996
7,8,15575,86,15548,0.998
8,9,15575,86,15561,0.999
9,10,15575,86,15567,0.999


In [16]:
print("TFIDF Validate")
tfidf_val_results = []
for i in range(10):
    json_i = accuracy(tfidf_val, i+1)
    tfidf_val_results.append(json_i)
import pandas as pd 
tfidf_val_df = pd.read_json(json.dumps(tfidf_val_results))
tfidf_val_df

TFIDF Validate


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1871,4,1452,0.776
1,2,1871,4,1677,0.896
2,3,1871,4,1763,0.942
3,4,1871,4,1814,0.970
4,5,1871,4,1845,0.986
5,6,1871,4,1860,0.994
6,7,1871,4,1865,0.997
7,8,1871,4,1868,0.998
8,9,1871,4,1868,0.998
9,10,1871,4,1869,0.999


In [17]:
print("TFIDF Test")
tfidf_test_results = []
for i in range(10):
    json_i = accuracy(tfidf_test, i+1)
    tfidf_test_results.append(json_i)
import pandas as pd 
tfidf_test_df = pd.read_json(json.dumps(tfidf_test_results))
tfidf_test_df

TFIDF Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1345,0.791
1,2,1701,3,1535,0.902
2,3,1701,3,1617,0.951
3,4,1701,3,1656,0.974
4,5,1701,3,1684,0.990
5,6,1701,3,1693,0.995
6,7,1701,3,1697,0.998
7,8,1701,3,1699,0.999
8,9,1701,3,1700,0.999
9,10,1701,3,1701,1.000


# Custom TFIDF

In [18]:
import math 
class TFIDF:


    def __init__(self, tf_scheme="raw_count", idf_scheme="idf", k=0.5):
        self.tf_scheme = tf_scheme
        self.idf_scheme = idf_scheme
        self.k = k

    def fit(self, corpus):
        """
        Fit the various statistics that are required to calculate BM25 ranking
        score using the corpus given.

        Parameters
        ----------
        corpus : list[list[str]]
            Each element in the list represents a document, and each document
            is a list of the terms.

        Returns
        -------
        self
        """
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            # idf[term] = math.log(corpus_size / (freq))
            idf[term] = math.log(corpus_size / (freq + 1))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self
        
    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query, index):
        score = 0.0

        # Choosing weighting scheme for tf
        tf_ = self.tf_
        if (self.tf_scheme == "raw_count"):
            tf_ = self.tf_
        elif (self.tf_scheme == "binary"):
            tf_ = [{key:1 for (key, value) in tf.items()} for tf in self.tf_]
        elif (self.tf_scheme == "term_frequency"):
            tf_ = [{key:value/sum(tf.values()) for (key, value) in tf.items()} for tf in self.tf_]

        # Choosing weighting scheme for idf
        idf_ = self.idf_
        if (self.idf_scheme == "unary"):
            idf_ = {key:1 for (key, value) in self.idf_.items()}
        if (self.idf_scheme == "idf"):
            idf_ = self.idf_

        frequencies = tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            tf = frequencies[term]
            idf = idf_[term]
            score += tf * idf 

        return score

In [19]:
def get_topk_sentences_custom(data, top_k, scorer):
    copied_data = deepcopy(data)
    for topic in copied_data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            context = [[word.lower() for word in word_tokenize(cxt)] for cxt in context]
            scorer.fit(context)
            for qa in paragraph['qas']:
                question = [word.lower() for word in word_tokenize(qa['question'])]
                scores = torch.Tensor(scorer.search(question))

                k = min(top_k, len(context))
                top_results = torch.topk(scores, k=k)
                qa['candidate_indices'] = top_results[1].tolist()
                qa['candiate_scores'] = top_results[0].tolist()
    return copied_data

In [20]:
tfidf = TFIDF("raw_count", "idf")

In [21]:
tfidf_train = get_topk_sentences_custom(train_json, 10, tfidf)
tfidf_val = get_topk_sentences_custom(val_json, 10, tfidf)
tfidf_test = get_topk_sentences_custom(test_json, 10, tfidf)

In [22]:
print("TFIDF Train")
tfidf_train_results = []
for i in range(10):
    json_i = accuracy(tfidf_train, i+1)
    tfidf_train_results.append(json_i)
import pandas as pd 
tfidf_train_df = pd.read_json(json.dumps(tfidf_train_results))
tfidf_train_df

TFIDF Train


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,15575,86,11047,0.709
1,2,15575,86,13536,0.869
2,3,15575,86,14537,0.933
3,4,15575,86,15035,0.965
4,5,15575,86,15299,0.982
5,6,15575,86,15428,0.991
6,7,15575,86,15499,0.995
7,8,15575,86,15537,0.998
8,9,15575,86,15556,0.999
9,10,15575,86,15566,0.999


In [23]:
print("TFIDF Validate")
tfidf_val_results = []
for i in range(10):
    json_i = accuracy(tfidf_val, i+1)
    tfidf_val_results.append(json_i)
import pandas as pd 
tfidf_val_df = pd.read_json(json.dumps(tfidf_val_results))
tfidf_val_df

TFIDF Validate


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1871,4,1341,0.717
1,2,1871,4,1623,0.867
2,3,1871,4,1742,0.931
3,4,1871,4,1798,0.961
4,5,1871,4,1835,0.981
5,6,1871,4,1851,0.989
6,7,1871,4,1859,0.994
7,8,1871,4,1866,0.997
8,9,1871,4,1866,0.997
9,10,1871,4,1867,0.998


In [24]:
print("TFIDF Test")
tfidf_test_results = []
for i in range(10):
    json_i = accuracy(tfidf_test, i+1)
    tfidf_test_results.append(json_i)
import pandas as pd 
tfidf_test_df = pd.read_json(json.dumps(tfidf_test_results))
tfidf_test_df

TFIDF Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1249,0.734
1,2,1701,3,1505,0.885
2,3,1701,3,1609,0.946
3,4,1701,3,1655,0.973
4,5,1701,3,1676,0.985
5,6,1701,3,1689,0.993
6,7,1701,3,1690,0.994
7,8,1701,3,1699,0.999
8,9,1701,3,1700,0.999
9,10,1701,3,1701,1.000


# Tuning

BM25 Tuning

In [25]:
from rank_bm25 import BM25Okapi
from rank_bm25 import BM25Plus
from rank_bm25 import BM25L
def get_topk_sentences_bm25_(data, top_k, k1=1.5, b=0.75):
    """
    Using sklearn TFIDF to get top k most relevant sentences for each question
    """
    copied_data = deepcopy(data)
    for topic in copied_data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            vectorizer = TfidfVectorizer()
            tokenizer = vectorizer.build_tokenizer()
            tokenized_context = [tokenizer(sentence.lower()) for sentence in context]
            
            # learn statistic of the context sentences.
            bm25 = BM25Plus(tokenized_context, k1=k1, b=b)
            for qa in paragraph['qas']:
                question = qa['question']
                query = tokenizer(question)

                # compute scores of each sentence in the context.
                scores = bm25.get_scores(query)
                k = min(top_k, len(context))
                top_results = torch.topk(torch.tensor(scores.reshape(-1)), k=k)
                qa['candidate_indices'] = top_results[1].tolist()
    return copied_data

In [26]:
bm25_data_ = []
for k1 in np.arange(0, 2.25, 0.25):
    temp = []
    for b in np.arange(0, 1.1, 0.1):
        k1_ = k1 
        b_ = b 
        temp.append(get_topk_sentences_bm25_(val_json, 1, k1_, b_))
    bm25_data_.append(temp)

/home/thanhdo/anaconda3/envs/former/lib/python3.9/site-packages/rank_bm25.py:193: RuntimeWarning: invalid value encountered in true_divide
  score += (self.idf.get(q) or 0) * (self.delta + (q_freq * (self.k1 + 1)) /


In [27]:
results = [] 
for bm25_k1 in bm25_data_:
    temp = []
    for bm25_b in bm25_k1:
        json_i = accuracy(bm25_b, 1).get("accuracy")
        temp.append(json_i)
    results.append(temp)
results_df = pd.read_json(json.dumps(results))
# results_df.append(pd.Series(np.arange(0, 1.1, 0.1)), ignore_index=True)
results_df['k1'] = np.arange(0, 2.25, 0.25)
results_df

,0,1,2,3,4,5,6,7,8,9,10,k1
0,0.192,0.192,0.192,0.192,0.192,0.192,0.192,0.192,0.192,0.192,0.192,0.00
1,0.738,0.738,0.738,0.741,0.742,0.744,0.743,0.746,0.745,0.746,0.735,0.25
2,0.739,0.740,0.742,0.743,0.746,0.747,0.749,0.751,0.748,0.751,0.740,0.50
3,0.738,0.739,0.742,0.743,0.749,0.751,0.749,0.751,0.751,0.752,0.740,0.75
4,0.738,0.739,0.742,0.745,0.749,0.751,0.751,0.752,0.751,0.749,0.738,1.00
5,0.737,0.739,0.742,0.744,0.750,0.753,0.753,0.752,0.748,0.748,0.736,1.25
6,0.738,0.739,0.741,0.743,0.750,0.753,0.753,0.749,0.747,0.747,0.735,1.50
7,0.737,0.738,0.741,0.745,0.749,0.753,0.753,0.748,0.747,0.748,0.734,1.75
8,0.737,0.737,0.741,0.745,0.749,0.753,0.753,0.748,0.748,0.746,0.732,2.00


In [28]:
print("BM25 Test")
bm25_test = get_topk_sentences_bm25_(test_json, 10, 1.5, 0.5)
bm25_test_results = []
for i in range(10):
    json_i = accuracy(bm25_test, i+1)
    bm25_test_results.append(json_i)
import pandas as pd 
bm25_test_df = pd.read_json(json.dumps(bm25_test_results))
bm25_test_df

BM25 Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1307,0.768
1,2,1701,3,1503,0.884
2,3,1701,3,1602,0.942
3,4,1701,3,1646,0.968
4,5,1701,3,1676,0.985
5,6,1701,3,1687,0.992
6,7,1701,3,1692,0.995
7,8,1701,3,1698,0.998
8,9,1701,3,1700,0.999
9,10,1701,3,1701,1.000


### TFIDF Tuning

In [29]:
tf_schemes = ["raw_count", "binary", "term_frequency"]
idf_schmes = ["unary", "idf"]

results = []
for tf_scheme in tf_schemes:
    temp = []
    for idf_scheme in idf_schmes:
        tfidf = TFIDF(tf_scheme, idf_scheme)
        tfidf_val = get_topk_sentences_custom(val_json, 1, tfidf)
        temp.append(accuracy(tfidf_val, 1).get("accuracy"))
    results.append(temp)

In [30]:
tfidf_val_df = pd.DataFrame(results)
tfidf_val_df

,0,1
0,0.673,0.718
1,0.729,0.742
2,0.654,0.715


In [31]:
tfidf = TFIDF("binary", "idf")
tfidf_val = get_topk_sentences_custom(test_json, 10, tfidf)

print("TFIDF Test")
tfidf_test_results = []
for i in range(10):
    json_i = accuracy(tfidf_val, i+1)
    tfidf_test_results.append(json_i)
import pandas as pd 
tfidf_test_df = pd.read_json(json.dumps(tfidf_test_results))
tfidf_test_df

TFIDF Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1291,0.759
1,2,1701,3,1520,0.894
2,3,1701,3,1609,0.946
3,4,1701,3,1653,0.972
4,5,1701,3,1674,0.984
5,6,1701,3,1687,0.992
6,7,1701,3,1690,0.994
7,8,1701,3,1698,0.998
8,9,1701,3,1700,0.999
9,10,1701,3,1701,1.000


# Investigate

In [32]:
def inspect(data, top_k):
    """
    Calculate accuracy 
    Given queries Q = {Q1, ..., Qm} 
    Document D = {S1, ..., Sn}
    The sentence containing the answer to the question Qi is Ai
    
    Acc@K = 1/|Q| * sum(is Ai in get_top_k(Q, D))
    """
    results = []
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0:
                    if len(sent_tokenize(qa['answers'][0]['text'])) > 1: break 
                    start_answer = qa['answers'][0]['answer_start']
                    k = min(top_k, len(qa['candidate_indices']))
                    ans_sent_idx = find_sentence(context, start_answer)
                    flag = ans_sent_idx in qa['candidate_indices'][:k]
                    results.append(flag)
                    if (flag == False):
                        print(qa['question'])
                        print(qa['answers'][0]['text'])
                        print("True sentence:", context[ans_sent_idx])
                        print(context)
                        print(ans_sent_idx, start_answer)
                        print("Best candidates: \n" + context[qa['candidate_indices'][0]])
                        print("")

    # return {"top_k":top_k, "total":len(results), "true":sum(results), "accuracy":round(sum(results) / len(results), 3)}

inspect(tfidf_val, 1)

Độ cao tối đa của tầng đối lưu là bao nhiêu?
20 km
True sentence: Tầng đối lưu bắt đầu từ bề mặt Trái Đất mở rộng ra đến cao độ 20 km (12 dặm) ở các vùng nhiệt đới, giảm tới khoảng 11 km ở các vĩ độ trung bình, ít hơn 7 km (4 dặm) ở các vùng cực về mùa hè còn trong mùa đông là không rõ ràng.
['Tầng đối lưu bắt đầu từ bề mặt Trái Đất mở rộng ra đến cao độ 20 km (12 dặm) ở các vùng nhiệt đới, giảm tới khoảng 11 km ở các vĩ độ trung bình, ít hơn 7 km (4 dặm) ở các vùng cực về mùa hè còn trong mùa đông là không rõ ràng.', 'Lớp khí quyển này chiếm khoảng 80% tổng khối lượng của toàn bộ khí quyển, gần như toàn bộ hơi nước và xon khí (aerosol).', 'Trong khu vực tầng đối lưu thì không khí liên tục luân chuyển và tầng này là tầng có mật độ không khí lớn nhất của khí quyển Trái Đất.', 'Nitơ và ôxy là các chất khí chủ yếu có mặt trong tầng này.', 'Tầng đối lưu nằm ngay phía dưới tầng bình lưu.', 'Phần thấp nhất của tầng đối lưu, nơi ma sát với bề mặt Trái Đất ảnh hưởng tới luồng không khí, là lớp

In [33]:
corpus_ = ['Tuy nhiên sự thật các cải cách của ông kéo theo các biện pháp áp bức.', 'Ông nắm quyền khi mà ngoài Sài Gòn và các vùng phụ cận, miền Nam Việt Nam được phân chia thành các vùng đất do Việt Minh kiểm soát và lãnh địa các giáo phái "thần quyền" như Cao Đài và Hòa Hảo.', 'Ông chống lại bất đồng chính kiến, và đòi hỏi lòng trung thành tuyệt đối cá nhân với các quan chức hàng đầu.', 'Tài liệu của Mỹ cũng cho biết Ngô Đình Diệm là một người Công giáo bảo thủ.', 'Ngô Đình Diệm lúc đầu đã được chào đón nồng nhiệt trong một số người từng là thành viên Việt Minh.', 'Nhưng chương trình cải cách điền địa của ông thất bại và dừng lại năm 1959.']
sum_ = sum([len(sent) for sent in corpus_]) + 5
find_sentence(corpus_, sum_), len(corpus_)

(6, 6)

In [34]:
def inspect2(data):
    num_qa = 0
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0 and len(sent_tokenize(qa['answers'][0]['text'])) == 1:
                    answer = qa['answers'][0]['text']
                    answer_start = qa['answers'][0]['answer_start']
                    ans_sent_idx = find_sentence(context, answer_start)
                    # print(ans_sent_idx)
                    if (answer in context[ans_sent_idx]):
                        print(qa['question'])
                        print(answer)
                        print(context[ans_sent_idx])
                        print(" ")
                    else:
                        num_qa += 1
    return num_qa

inspect2(val_json)

Vào giai đoạn nào thì khu vực lưu vực Lạc Hà có sự tồn tại của văn hóa Bùi Lý Cương?
khoảng thời gian 7000 TCN - 5000 TCN
Vào đầu thời đại đồ đá mới, tại khu vực trung du Hoàng Hà đã có nhiều thể loại hình thức văn hóa phát sinh và phát triển, thông qua khảo cổ, đã phát hiện được văn hóa Bùi Lý Cương tồn tại trong khoảng thời gian 7000 TCN - 5000 TCN ở lưu vực Lạc Hà, khu định cư Gia Hồ từng tồn tại từ 7000 TCN - 5800 TCN ở khu vực huyện Vũ Dương ngày nay.
 
Vào khoảng thời gian 7000 TCN - 5800 TCN tại huyện Vũ Dương ngày nay đã có sự tồn tại của khu định cư nào?
khu định cư Gia Hồ
Vào đầu thời đại đồ đá mới, tại khu vực trung du Hoàng Hà đã có nhiều thể loại hình thức văn hóa phát sinh và phát triển, thông qua khảo cổ, đã phát hiện được văn hóa Bùi Lý Cương tồn tại trong khoảng thời gian 7000 TCN - 5000 TCN ở lưu vực Lạc Hà, khu định cư Gia Hồ từng tồn tại từ 7000 TCN - 5800 TCN ở khu vực huyện Vũ Dương ngày nay.
 
Loài thực vật nào mà chỉ có tại Gia Hồ trồng mà các nơi khác của văn h

2

In [35]:
def inspect3(data, top_k):
    """
    Calculate accuracy 
    Given queries Q = {Q1, ..., Qm} 
    Document D = {S1, ..., Sn}
    The sentence containing the answer to the question Qi is Ai
    
    Acc@K = 1/|Q| * sum(is Ai in get_top_k(Q, D))
    """
    results = []
    lens = []
    invalid = 0 # number of invalid qa
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for cxt in context:
                lens.append(len(word_tokenize(cxt)))
            idxs = []
            # for qa in paragraph['qas']:
            #     if len(qa['answers']) > 0:

            #         # If the answer is from multiple sentences then discard this qa
            #         if len(sent_tokenize(qa['answers'][0]['text'])) > 1:
            #             invalid += 1
            #             continue 

            #         start_answer = qa['answers'][0]['answer_start']
            #         # k = min(top_k, len(qa['candidate_indices']))

            #         # Find the index of sentence that contains the answer 
            #         ans_sent_idx = find_sentence(context, start_answer)
            #         idxs.append(ans_sent_idx)
            # results.append(idxs)
    return lens 

idxs = inspect3(train_json, 5)

In [36]:
lens_val = inspect3(val_json, 4)

In [37]:
train_json['data'][1]['paragraphs'][0]['context']

['Pháp (tiếng Pháp: France; phát âm địa phương: [fʁɑ̃s]), tên chính thức là nước Cộng hòa Pháp (tiếng Pháp: République française [ʁepyblik fʁɑ̃sɛz]), là một quốc gia có lãnh thổ chính nằm tại Tây Âu cùng một số vùng và lãnh thổ hải ngoại.',
 'Phần lãnh thổ Pháp tại châu Âu trải dài từ Địa Trung Hải đến eo biển Manche và biển Bắc, và từ sông Rhin đến Đại Tây Dương.',
 'Pháp còn có Guyane thuộc Pháp trên đại lục Nam Mỹ cùng một số đảo tại Đại Tây Dương, Thái Bình Dương và Ấn Độ Dương.',
 '18 vùng của Pháp có tổng diện tích 643.801 km², dân số tính đến tháng 1 năm 2017 là gần 67 triệu người.',
 'Pháp là nước cộng hòa bán tổng thống nhất thể, thủ đô Paris cũng là thành phố lớn nhất, trung tâm văn hóa và thương mại chính của quốc gia.',
 'Các trung tâm đô thị lớn khác là Marseille, Lyon, Lille, Nice, Toulouse và Bordeaux.']